### Import Libraries

In [2]:
import psycopg2
import psycopg2.extras
import configparser as configparser
import pandas as pd

## Fetching new_relation_movie table

### Funtion: To parse INI file

In [3]:
def parse_ini(section: str) -> dict:
    """
    This function parses ini file for configuration details
    :param section: section to read from ini
    :return: Dictionary of config details
    """
    config = dict()
    parser = configparser.ConfigParser()
    parser.read("imdb_database.ini")
    if parser.has_section(section):
        config_items = parser.items(section)
        for item in config_items:
            config[item[0]] = item[1]
    return config

In [4]:
db_config = parse_ini("postgresql")
db_config
table_name = "new_relation_movie"

### Function: To connect to IMDB databae and run select query to fetch new_relation_movie table into new_relation_df

In [5]:
with psycopg2.connect(**db_config) as conn:
    try:
        query = "SELECT * FROM " + table_name
        new_relation_df = pd.read_sql_query(query, conn)
        print("The query results loaded to Dataframe")
    except (Exception, psycopg2.DatabaseError) as error:
        print("SQL Exception:" + str(error))

C:\Users\anura\AppData\Local\Temp\ipykernel_47676\412817476.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_relation_df = pd.read_sql_query(query, conn)


The query results loaded to Dataframe


In [6]:
# new_relation_df = new_relation_df.drop('id', axis=1)
new_relation_df

,id,movieid,type,startyear,runtime,avgrating,genreid,genre,memberid,birthyear,character
0,1,147,movie,1897.0,100,5.2,1,Documentary,179163,1866.0,52
1,2,147,movie,1897.0,100,5.2,6,Sport,179163,1866.0,52
2,3,147,movie,1897.0,100,5.2,7,News,179163,1866.0,52
3,4,147,movie,1897.0,100,5.2,1,Documentary,280615,1863.0,52
4,5,147,movie,1897.0,100,5.2,6,Sport,280615,1863.0,52
...,...,...,...,...,...,...,...,...,...,...,...
1839663,1839664,26658277,tvSeries,2009.0,120,NaN,25,Talk-Show,14538771,NaN,55
1839664,1839665,26658688,tvSeries,2007.0,240,NaN,25,Talk-Show,1715086,1972.0,55
1839665,1839666,26658688,tvSeries,2007.0,240,NaN,25,Talk-Show,1760063,NaN,55
1839666,1839667,26658688,tvSeries,2007.0,240,NaN,25,Talk-Show,8386585,NaN,55


### Attributes in new_relation_movie table

In [7]:
attributes = list(new_relation_df.columns)
attributes

['id',
 'movieid',
 'type',
 'startyear',
 'runtime',
 'avgrating',
 'genreid',
 'genre',
 'memberid',
 'birthyear',
 'character']

## Naive approach to find FDs

### Combinations of all atributes

In [8]:
from itertools import chain, combinations

dependencies = []
powerset_attributes = list(chain.from_iterable(combinations(attributes, a) for a in range(2, len(attributes)+1)))
powerset_attributes = [list(a) for a in powerset_attributes]
powerset_attributes = [[a] for a in attributes] + powerset_attributes

### Function: To check Dependency using partition refinement
It checks the if the count of equivalence classes for set LHS and count of equivalence classes for set (LHS v {RHS}) is same
(Simplified partition refinement)

In [9]:
def check_dependency(dataframe, lhs, rhs) -> bool:
    count1 = dataframe[list(lhs)].drop_duplicates().shape[0]
    lhs_rhs = list(lhs) + [rhs]
    count2 = dataframe[list(lhs_rhs)].drop_duplicates().shape[0]
    if count1 != count2:
        return False
    return True

### Brute force checking all the possibilities
Restricted to first 121 combinations

In [10]:
for lhs in powerset_attributes[:10]:
    for rhs in attributes:
        if rhs not in lhs:
            if check_dependency(new_relation_df, lhs, rhs):
                dependencies.append([tuple(lhs), rhs])
len(dependencies)

17

In [ ]:
dependencies

## Pruning approach to find FDs

### Initializing

In [12]:
dependencies = []
candidate_plus_RHS = dict()  # To hold RHS candidates for a given set X
previous_level_alphas = [tuple([])]  # level 0
candidate_plus_RHS[tuple([])] = set(attributes)  # for level 0
candidate_plus_RHS

{(): {'avgrating',
  'birthyear',
  'character',
  'genre',
  'genreid',
  'id',
  'memberid',
  'movieid',
  'runtime',
  'startyear',
  'type'}}

### Function: To calculate RHS Candidates for a given set X subset of Attributes R of relation

In [13]:
def get_candidate_plus_RHS(alpha, previous_level_alphas):
    # print(set(alpha))
    value = set(attributes)
    # print(previous_level_alphas)
    for previous_alpha in previous_level_alphas:
        # print(set(previous_alpha))
        if set(previous_alpha).issubset(set(alpha)):
            value = value.intersection(candidate_plus_RHS[tuple(sorted(previous_alpha))])
    return value

### Check if set X is a superkey for the new_relation_movie

In [14]:
def is_superkey(alpha):
    count = new_relation_df[list(alpha)].drop_duplicates().shape[0]
    if count == new_relation_df.shape[0]:
        return True
    return False

### Pruning Algorithm to get functional dependencies

In [17]:
current_level_alphas = list(combinations(attributes, 1))  # All Alpha's at level '1' in lattice
for level in range(1, 4):
    print(current_level_alphas)
    # Initializing Candidate RHS for all Alpha's
    for alpha in current_level_alphas:
        alpha = tuple(sorted(alpha))
        candidate_plus_RHS[alpha] = get_candidate_plus_RHS(alpha, previous_level_alphas)
    # Dependency checking
    for alpha in current_level_alphas:
        alpha = tuple(sorted(alpha))
        # Candidate RHS should be a subset(in domain) of Alpha
        possible_rhs = set(alpha).intersection(candidate_plus_RHS[alpha])
        for rhs in list(possible_rhs):
            lhs = list(set(alpha).difference({rhs}))  # Alpha\{RHS} -> RHS
            if lhs == None or len(lhs) <= 0:
                continue
            if check_dependency(new_relation_df, lhs, rhs):
                dependencies.append([tuple(lhs), rhs])
                # RHS Candidate Pruning
                # Remove rhs
                candidate_plus_RHS[alpha].remove(rhs)
                # Remove all attributes that doesn't belong to Alpha
                R_alpha = set(attributes).difference(set(alpha))
                candidate_plus_RHS[alpha] = candidate_plus_RHS[alpha].difference(R_alpha)
    # Pruning
    prune_alphas = []
    for alpha in current_level_alphas:
        alpha = tuple(sorted(alpha))
        if not bool(candidate_plus_RHS[alpha]):  # if Candidate RHS is empty
            prune_alphas.append(set(alpha))
        if is_superkey(alpha): # Add all depedencies possible as it is superkey
            for A in list(candidate_plus_RHS[alpha].difference(set(alpha))):
                reduction = set(attributes)
                for B in list(alpha):
                    x = [i for i in set(alpha).union({A}).difference({B})]
                    x = sorted(x)
                    # print(tuple(x))
                    reduction.intersection(candidate_plus_RHS[tuple(x)])
                if A in reduction:
                    dependencies.append([alpha, A])
            prune_alphas.append(set(alpha))
    # Generating Alpha's for next level
    previous_level_alphas = current_level_alphas
    current_level_alphas = list(combinations(attributes, level))
    for alpha in current_level_alphas:
        for pruned in prune_alphas:
            if set(alpha).issuperset(pruned):
                current_level_alphas.remove(alpha)
                break


[('id',), ('movieid',), ('type',), ('startyear',), ('runtime',), ('avgrating',), ('genreid',), ('genre',), ('memberid',), ('birthyear',), ('character',)]
[('movieid',), ('type',), ('startyear',), ('runtime',), ('avgrating',), ('genreid',), ('genre',), ('memberid',), ('birthyear',), ('character',)]
[('id', 'movieid'), ('id', 'type'), ('id', 'startyear'), ('id', 'runtime'), ('id', 'avgrating'), ('id', 'genreid'), ('id', 'genre'), ('id', 'memberid'), ('id', 'birthyear'), ('id', 'character'), ('movieid', 'type'), ('movieid', 'startyear'), ('movieid', 'runtime'), ('movieid', 'avgrating'), ('movieid', 'genreid'), ('movieid', 'genre'), ('movieid', 'memberid'), ('movieid', 'birthyear'), ('movieid', 'character'), ('type', 'startyear'), ('type', 'runtime'), ('type', 'avgrating'), ('type', 'genreid'), ('type', 'genre'), ('type', 'memberid'), ('type', 'birthyear'), ('type', 'character'), ('startyear', 'runtime'), ('startyear', 'avgrating'), ('startyear', 'genreid'), ('startyear', 'genre'), ('start

In [ ]:
minimal_dependencies = set()
for x in dependencies:
    minimal_dependencies.add(tuple(x))
minimal_dependencies
